In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Задача 01 - Word2Vec Model Lite ##

Write a program that trains Word2Vec model. Do not use print() instructions in your code, otherwise test procedure will not succeed; the message "Wrong Answer" indicates answer format is incorrect (print() in the code, missing words in the dictionary, etc.). The message "Embeddings are not good enough" means you're on the right track and you should focus on the model improvement. In this version of the assignment the checks on the embeddings are easier.

You may think of the input string as being pre-processed with the following function:

 '''import re

import string

def clean(inp: str) -> str:

    inp = inp.translate(str.maketrans(string.punctuation, " "*len(string.punctuation)))

    inp = re.sub(r'\s+', ' ', inp.lower())

    return inp

'''

I.e. given the input "Your string!" the output will be "your string ".


Input: data (string) - cleaned documents without punctuation in one line
Output: w2v_dict (dict: key (string) - a word from vocabulary, value (numpy array) - the word's embedding)

Time limit: 25 seconds
Memory limit: 128 MB

In [2]:
import re
import string

def clean(inp: str) -> str:
    
    # string.punctuation -> '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    # заменяем вышеперечисленную пункуацию на пробелы
    inp = inp.translate(str.maketrans(string.punctuation, " "*len(string.punctuation)))
    # заменяем множественные пробелы на один пробел и приводим в нижний регистр
    inp = re.sub(r'\s+', ' ', inp.lower())
    return inp

In [3]:
sample = "\n".join(['Как сказал Дэвид Лоуренс, «любая полуправда в конце концов приходит к самоотрицанию в противоположной полуправде».',
 'Если я вижу только мысленную карту первой альтернативы — собственную неполную карту, то, с одной стороны, единственный способ решить проблему — убедить вас изменить вашу парадигму или даже заставить вас принять мою точку зрения, альтернативную вашей.',
 'Это еще и единственный способ сохранить свою «Я-концепцию»: я должен победить, вы — проиграть.',
 'С другой стороны, отбросив собственную карту и следуя вашей — второй альтернативе, я сталкиваюсь с той же проблемой.',
 'Вы не можете гарантировать, что ваша мысленная карта сколько-нибудь полна, и я рискую заплатить чудовищную цену, руководствуясь ею.'])
clean_str = clean(sample)
clean_str

'как сказал дэвид лоуренс «любая полуправда в конце концов приходит к самоотрицанию в противоположной полуправде» если я вижу только мысленную карту первой альтернативы — собственную неполную карту то с одной стороны единственный способ решить проблему — убедить вас изменить вашу парадигму или даже заставить вас принять мою точку зрения альтернативную вашей это еще и единственный способ сохранить свою «я концепцию» я должен победить вы — проиграть с другой стороны отбросив собственную карту и следуя вашей — второй альтернативе я сталкиваюсь с той же проблемой вы не можете гарантировать что ваша мысленная карта сколько нибудь полна и я рискую заплатить чудовищную цену руководствуясь ею '

**Краткие выводы:** Видим что предложенный вариант предобраотки заменяет не всю пунктуаци, как минимум сотаются символы «»— и возможно другие, как следствие, на всякий случай необходимо токенизировать на основе регулярных выражений, соответсвующих возможным словам (возможно входная строка будет на английском языке, и не будет содержать незатронутой пунктуации, но лучше всего дополнительно почистить)

## Решение ##

- попробовать обучать на нормальном тексте (можно на английском) после чего:
  - посмотреть как падает лосс
  - на практике посмотреть схожесть токенов
- попробовать убирать стоп слова (при этом проверить на голых эмбеддингах и легком задании, допустимо ли это) 
- разные размеры эмбеддингов (50, 100, 200, 300)
- инициализация эмбеддингов шумом определенного типа
- разный размер окна (какой рекомендуют? какой использовался в предыдущем курсе?) ==> использовался радиут 5, т.е. полный размер 11, при этом обучение проходил полностью от начала и до конца, т.е. позиции когда окно не помещается полностью (остается контект только справа/слева, тоже попадали в обучение)
- разный размер негатив сэмплов (какой рекомендуют? какой использовался в предыдущем курсе?)
- посмотреть lr который использовался в предыдущем курсе ==> использовался 1e-2, но так же был шедулер)
- делать градиентный шаг не сразу, а как бы через батч сайз
- на выходе берем контектстные эмбеддинги или центральные
- прочитать статьи (которая в чате) и которая рекомендована в лекции
- возможно пробежаться по алгоритм, который использовался в предыдущем курсе
- если совсем все плохо, и уже будет поджимать время или попытки, попробовать алгоритм из предыдущего курса, а так же написать тому, кто вел переписку в чате

In [4]:
import numpy as np
import re
import torch
from torch import nn
from torch.nn import functional as F

class Tokenizer:
    def __init__(self, text, word_pattern=r"[a-zа-яё0-9_]+"):
        self.re_word_pattern = re.compile(word_pattern)
        list_vocab = list(set(self.re_word_pattern.findall(text.lower())))
        self.vocab_size = len(list_vocab)
        self.vocab_id2word = dict(zip(range(len(list_vocab)), list_vocab))
        self.vocab_word2id = dict(zip(list_vocab, range(len(list_vocab))))
    
    def tokenize(self, text):
        return list(map(lambda x: self.vocab_word2id[x], 
                        self.re_word_pattern.findall(text.lower())))
    
    def untokenize(self, list_ids):
        return list(map(lambda x: self.vocab_id2word[x], list_ids))


class Word2vec(nn.Module):
    
    def __init__(self, vocab_size: int, emb_dim: int, negative_size = 20):
        super().__init__()
        self.vocab_size = vocab_size
        self.center_emb = torch.nn.Embedding(vocab_size, emb_dim)
        self.context_emb = torch.nn.Embedding(vocab_size, emb_dim)
        self.negative_size = negative_size
        
    def forward(self, x_window: list):
        center_id = len(x_window) // 2  # center index in list x_window 
        
        center_word = self.center_emb(torch.LongTensor([x_window[center_id]])).flatten()
        context_words = self.context_emb(torch.LongTensor(x_window[:center_id] + 
                                                          x_window[center_id + 1:]))
        context_noise_words = self.context_emb(torch.randint(0, 
                                                             self.vocab_size, 
                                                             (self.negative_size,)))
        
        positive_score = F.logsigmoid(context_words @ center_word)
        negative_score = F.logsigmoid(-(context_noise_words @ center_word))
        
        return positive_score.sum() + negative_score.sum()
    

NUM_EPOCH = 20
EMB_DIM = 50
NEGATIVE_SIZE = 20

def train(data: str):
    """
    return: w2v_dict: dict
            - key: string (word)
            - value: np.array (embedding)
    """
    my_tokenizer = Tokenizer(data)
    word2vec_model = Word2vec(my_tokenizer.vocab_size, EMB_DIM, NEGATIVE_SIZE)
    loss_func = lambda x: -x

    optimizer = torch.optim.AdamW(word2vec_model.parameters(), lr=0.01)

    window_size = 9

    data_tokens = my_tokenizer.tokenize(data)

    for n_epoch in range(NUM_EPOCH):
        epoch_loss = 0.
        counts = len(data_tokens) - window_size + 1
        for position in range(counts):
            optimizer.zero_grad()
            loss = loss_func(word2vec_model(data_tokens[position:position + window_size]))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        #print(epoch_loss / counts)
        
    return dict(zip(my_tokenizer.vocab_word2id.keys(),
                    word2vec_model.context_emb.weight.data.numpy()))

In [5]:
train(clean_str);